In [23]:
from utils import quantise_tensor_flat, dequantise_tensor_flat, compute_quantisation_mse,get_tensor_memory_size
import torch

t = torch.rand((10, 100))
chunk_size = 256
t_q, scales, locations = quantise_tensor_flat(t, chunk_size)
t_approx = dequantise_tensor_flat(t_q, scales, locations, chunk_size)

mse = compute_quantisation_mse(t.flatten(), t_approx)

print(f'Required memory: {get_tensor_memory_size(t)} vs {get_tensor_memory_size(t_q) + get_tensor_memory_size(scales) + get_tensor_memory_size(locations)}')

MSE: 0.0722302794456482
Norm: 18.426227569580078
RelativeMSE: 0.392%
Required memory: 4000 vs 1032
